In [41]:
import glob, json, pathlib
from tqdm.notebook import tqdm
sanitychecks = list(glob.glob(r"D:\ig_pipeline\cad\*\*\artifacts\sanitycheck.json"))
jsons = {}
for x in tqdm(sanitychecks):
    with open(x, "r") as f:
        name = "/".join(pathlib.Path(x).parts[-4:-2])
        jsons[name] = json.load(f)

  0%|          | 0/91 [00:00<?, ?it/s]

In [42]:
total = len(jsons)
failed = {name for name, x in jsons.items() if x["errors"]}
total, len(failed)

(91, 71)

In [50]:
from collections import defaultdict
import re
error_templates_and_ignore = {
    "Category (.*) for object .* does not exist on spreadsheet.": False,
    "(.*) has too many vertices: .* > .*": True,
    "(.*) link .* has different .* in instance .* compared to instance .*. .* difference: .*.": False,
    "(.*) is missing in instance .*, so relative transform check cannot be completed.": False,
    "(.*) has bad name.": False,
    "(.*) has negative object offset scale.*": False,
    "(.*) has scale that is not 1. Reset scale.": False,
    ".*No instance ID 0 instance of (.*)": False,
    "Articulated object (.*) instances have different scales for base links. This may have broken things during the match links script.": False,
    "All instances of (.*) do not have contiguous instance IDs. Missing: .*": False,
    "(.*) has disallowed type.*": False,
    "Light object (.*) should not have object offset rotation. Reset pivot.": False,
    "(.*) has different pivot offset position \(by (.*)\). Match pivots on each instance.": False,
    "(.*) has different pivot offset rotation \(by (.*)\). Match pivots on each instance.": False,
    "(.*) has different shear. Match scaling axes on each instance.": False,
    "Model ID (.*) contains 'todo'.": False,
    "(.*) should not have an upper side.": False,
    "(.*) should have an upper side.": False,
    "(.*) has different vertex count than recorded in provider.*": False,
    "(.*) has different face count than recorded in provider.*": False,
    "(.*) has no collision mesh. Create a collision mesh.": False,
    "(.*) has different UV unwrapping than recorded. Reunwrap the object.": False,
    "Inconsistent link sets within model (.*)": False,
    "(.*) is missing meta link: fillable.": False,
    "(.*) is missing meta link: heatsource.": False,
    "(.*) is missing meta link: togglebutton.": False,
    "(.*) is missing meta link: particleapplier.": False,
    "(.*) is missing meta link: fluidsource.": False,
    "(.*) is missing meta link: fluidsink.": False,
    "(.*) is missing meta link: togglebutton.": False,
    "(.*) element .* has too many vertices .*": False,
    "Model ID (.*) requires joints but has no joints.": False,
    "Expected meta links for (.*) are missing: .*": False,
    "Convex mesh fillable meta link (.*) should be of Editable Poly instead of .*": False,
    "Cloth object (.*) should consist of exactly 1 element. Currently it has .* elements.": False,
    "Cannot validate clothness: category (.*) not found in taxonomy.": False,
    "(.*) element .* is not a volume": False,
    "(.*) element .* has elements trimesh still finds splittable": False,
    "Object (.*) has .* fillable meshes. Should have no more than one.": False,
    "Model (.*) has category .* that is neither the from or to element in the rename file.": False,
    "(.*) has meta link not required by its synset: .*": True,
    "(.*) meta type .* ID .* has non-continuous subids .*": False,
    "Cannot validate meta links and joints for model ID (.*): Category .* not found in taxonomy.": False,
    "(.*) is in the deletion queue. Delete the object.": False,
    "(.*) has unapplied rename .*.": False,
    "fillable_seed is an invalid meta link under (.*).": False,
    "(.*) is not unique.": False,
    "Object (.*) has .* meshes. Should have no more than one.": False,
    "(.*) is not under another object but contains part tags {'connectedpart'}.": False,
}

SKIP_IGNORE = False

error_templates = list(error_templates_and_ignore.keys())
error_matches = defaultdict(set)
error_matches_by_file = defaultdict(lambda: defaultdict(set))
unknown_error_files = []
for name, f in jsons.items():
    unknown_error = False
    for error in f["errors"] + f["warnings"]:
        for i, tmpl in enumerate(error_templates):
            match = re.fullmatch(tmpl, error, re.S)
            if match is not None:
                if not (SKIP_IGNORE and error_templates_and_ignore[tmpl]):
                    error_matches[tmpl].add(match)
                    error_matches_by_file[tmpl][name].add(match)

                break
        else:
            print(error)
            unknown_error = True
    if unknown_error:
        unknown_error_files.append(name)
        
print("\nCounts")
for tmpl, objs in sorted(error_matches.items(), key=lambda x: len(x[1]), reverse=True):
    print(f"{tmpl}: {len(objs)}")

print("\n\nCounts by file")
for tmpl, matches_by_file in error_matches_by_file.items():
    all_matches = error_matches[tmpl]
    if not all_matches:
        continue
    print(f"\n{tmpl}")
    for name, matches in sorted(matches_by_file.items(), key=lambda x: x[1], reverse=True):
        if not matches:
            break
        print(f"  {name}: {len(matches)}")


Counts
(.*) has meta link not required by its synset: .*: 166
Articulated object (.*) instances have different scales for base links. This may have broken things during the match links script.: 140
(.*) has too many vertices: .* > .*: 123
Model ID (.*) requires joints but has no joints.: 98
Cloth object (.*) should consist of exactly 1 element. Currently it has .* elements.: 90
(.*) is missing meta link: fillable.: 88
(.*) link .* has different .* in instance .* compared to instance .*. .* difference: .*.: 85
(.*) has different pivot offset position \(by (.*)\). Match pivots on each instance.: 73
(.*) has scale that is not 1. Reset scale.: 57
(.*) has different UV unwrapping than recorded. Reunwrap the object.: 29
(.*) has no collision mesh. Create a collision mesh.: 22
Convex mesh fillable meta link (.*) should be of Editable Poly instead of .*: 10
(.*) element .* has too many vertices .*: 8
Cannot validate clothness: category (.*) not found in taxonomy.: 8
(.*) is in the deletion qu

In [44]:
from IPython.display import display, Markdown

# Print the number of kinds of error for each target
output = []
output.append("\n\n# Counts by error")
for name in jsons.keys():
  all_errors = {}
  for tmpl, matches_by_file in error_matches_by_file.items():
      if error_templates_and_ignore[tmpl]:
          continue
      matches = matches_by_file[name]
      if not matches:
          continue
      all_errors[tmpl] = matches

  if all_errors:
      output.append(f"\n## {name}")
      for k, v in sorted(all_errors.items(), key=lambda x: len(x[1]), reverse=True):
          output.append(f"  ### {k}: {len(v)}")
          for match in v:
            output.append("    " + match.group(0))
          output.append("")

display(Markdown("\n".join(output)))



# Counts by error

## objects/batch-00
  ### Model ID (.*) requires joints but has no joints.: 1
    Model ID gewjwh requires joints but has no joints.

  ### (.*) is missing meta link: fillable.: 1
    gewjwh is missing meta link: fillable.

  ### Model ID (.*) contains 'todo'.: 1
    Model ID jtodol contains 'todo'.


## objects/batch-01
  ### (.*) is missing meta link: fillable.: 1
    strtoe is missing meta link: fillable.


## objects/batch-02
  ### Model ID (.*) requires joints but has no joints.: 1
    Model ID otyngn requires joints but has no joints.

  ### (.*) is missing meta link: fillable.: 1
    buoocq is missing meta link: fillable.


## objects/batch-03
  ### Model ID (.*) requires joints but has no joints.: 2
    Model ID zvidlq requires joints but has no joints.
    Model ID qdnmwg requires joints but has no joints.

  ### Convex mesh fillable meta link (.*) should be of Editable Poly instead of .*: 1
    Convex mesh fillable meta link L-waffle_maker-yjmnej-0-Mfillable should be of Editable Poly instead of Cylinder


## objects/batch-04
  ### Model ID (.*) requires joints but has no joints.: 2
    Model ID wcqjew requires joints but has no joints.
    Model ID bqpmsv requires joints but has no joints.


## objects/batch-05
  ### (.*) is missing meta link: fillable.: 2
    svhjxd is missing meta link: fillable.
    svkdji is missing meta link: fillable.

  ### (.*) has no collision mesh. Create a collision mesh.: 2
    L-shelf-svhjxd-0-door3-base_link-R-lower has no collision mesh. Create a collision mesh.
    L-shelf-svhjxd-0-door2-base_link-R-lower has no collision mesh. Create a collision mesh.

  ### Convex mesh fillable meta link (.*) should be of Editable Poly instead of .*: 1
    Convex mesh fillable meta link plate-wgcgia-0-Mfillable should be of Editable Poly instead of Cone


## objects/batch-06
  ### Convex mesh fillable meta link (.*) should be of Editable Poly instead of .*: 2
    Convex mesh fillable meta link L-pizza_box-hvaucj-0-Mfillable should be of Editable Poly instead of Box
    Convex mesh fillable meta link L-tray-hjxczh-0-Mfillable should be of Editable Poly instead of Box


## objects/batch-07
  ### (.*) is missing meta link: particleapplier.: 1
    rwmotb is missing meta link: particleapplier.

  ### (.*) has different UV unwrapping than recorded. Reunwrap the object.: 1
    L-water_dispenser-rwmotb-0 has different UV unwrapping than recorded. Reunwrap the object.


## objects/batch-08
  ### (.*) has no collision mesh. Create a collision mesh.: 3
    lily-jobzwo-0 has no collision mesh. Create a collision mesh.
    lily-jtouad-0 has no collision mesh. Create a collision mesh.
    lily-jfeimu-0 has no collision mesh. Create a collision mesh.

  ### (.*) is missing meta link: fillable.: 1
    bnored is missing meta link: fillable.


## objects/batch-12
  ### Model ID (.*) requires joints but has no joints.: 1
    Model ID gejwoi requires joints but has no joints.


## objects/batch-13
  ### (.*) is not unique.: 1
    L-vase-nodcpg-0-Mfillable is not unique.

  ### Object (.*) has .* fillable meshes. Should have no more than one.: 1
    Object L-vase-nodcpg-0 has 2 fillable meshes. Should have no more than one.

  ### (.*) meta type .* ID .* has non-continuous subids .*: 1
    L-vase-nodcpg-0 meta type fillable ID 0 has non-continuous subids ['0', '0']


## objects/legacy_batch-00
  ### (.*) has no collision mesh. Create a collision mesh.: 1
    bottom_cabinet-kxushk-0-base_link has no collision mesh. Create a collision mesh.

  ### (.*) is not unique.: 1
    bottom_cabinet-pllcur-0-link_1-base_link-P-lower-Mcollision is not unique.

  ### (.*) meta type .* ID .* has non-continuous subids .*: 1
    bottom_cabinet-pllcur-0-link_1-base_link-P-lower meta type collision ID 0 has non-continuous subids ['0', '0']

  ### Object (.*) has .* meshes. Should have no more than one.: 1
    Object bottom_cabinet-pllcur-0-link_1-base_link-P-lower-Topenable has 2 collision meshes. Should have no more than one.


## objects/legacy_batch-03
  ### Model ID (.*) requires joints but has no joints.: 1
    Model ID qzwijs requires joints but has no joints.

  ### (.*) has no collision mesh. Create a collision mesh.: 1
    dishwasher-tnlzww-0-base_link has no collision mesh. Create a collision mesh.


## objects/legacy_batch-04
  ### Model ID (.*) requires joints but has no joints.: 1
    Model ID glzckq requires joints but has no joints.

  ### (.*) has no collision mesh. Create a collision mesh.: 1
    public_trash_can-pdmzhv-0-base_link has no collision mesh. Create a collision mesh.


## objects/legacy_batch-05
  ### (.*) is missing meta link: fillable.: 4
    nkxhvf is missing meta link: fillable.
    wuinhm is missing meta link: fillable.
    ptxdid is missing meta link: fillable.
    hxorzh is missing meta link: fillable.

  ### (.*) is not unique.: 1
    shelf-uriudf-0-base_link-Mopenfillable is not unique.

  ### (.*) meta type .* ID .* has non-continuous subids .*: 1
    shelf-uriudf-0-base_link meta type openfillable ID 0 has non-continuous subids ['0', '0']

  ### Object (.*) has .* meshes. Should have no more than one.: 1
    Object shelf-uriudf-0-base_link has 2 openfillable meshes. Should have no more than one.

  ### fillable_seed is an invalid meta link under (.*).: 1
    fillable_seed is an invalid meta link under shelf-wnvxme-0-base_link.


## objects/legacy_batch-06
  ### (.*) is missing meta link: fillable.: 4
    yfaufu is missing meta link: fillable.
    vbquye is missing meta link: fillable.
    ejooms is missing meta link: fillable.
    zexzrc is missing meta link: fillable.

  ### fillable_seed is an invalid meta link under (.*).: 1
    fillable_seed is an invalid meta link under furniture_sink-xiybkb-0-drawer0-base_link-P-lower.


## objects/legacy_batch-09
  ### (.*) is missing meta link: fillable.: 1
    acikzo is missing meta link: fillable.

  ### (.*) has no collision mesh. Create a collision mesh.: 1
    half_radish-zjeukz-0-base_link-Tsubpart has no collision mesh. Create a collision mesh.


## objects/legacy_batch-10
  ### (.*) is missing meta link: fillable.: 1
    ozykxk is missing meta link: fillable.


## objects/legacy_batch-11
  ### (.*) is missing meta link: fillable.: 1
    mcualj is missing meta link: fillable.

  ### Convex mesh fillable meta link (.*) should be of Editable Poly instead of .*: 1
    Convex mesh fillable meta link plate-iawoof-0-base_link-Mfillable should be of Editable Poly instead of Box


## objects/task_relevant-xy
  ### Convex mesh fillable meta link (.*) should be of Editable Poly instead of .*: 2
    Convex mesh fillable meta link plate-bgxzec-0-Mfillable should be of Editable Poly instead of Cylinder
    Convex mesh fillable meta link plate-qtfzeq-0-Mfillable should be of Editable Poly instead of Cylinder


## scenes/Beechwood_0_garden
  ### Articulated object (.*) instances have different scales for base links. This may have broken things during the match links script.: 6
    Articulated object spojpj instances have different scales for base links. This may have broken things during the match links script.
    Articulated object immwzb instances have different scales for base links. This may have broken things during the match links script.
    Articulated object qohxjq instances have different scales for base links. This may have broken things during the match links script.
    Articulated object lvgliq instances have different scales for base links. This may have broken things during the match links script.
    Articulated object ulnafj instances have different scales for base links. This may have broken things during the match links script.
    Articulated object dmwxyl instances have different scales for base links. This may have broken things during the match links script.

  ### Model ID (.*) requires joints but has no joints.: 1
    Model ID qesxhu requires joints but has no joints.

  ### (.*) is missing meta link: fillable.: 1
    qesxhu is missing meta link: fillable.

  ### Convex mesh fillable meta link (.*) should be of Editable Poly instead of .*: 1
    Convex mesh fillable meta link C-tray-zcmnji-0-Mfillable should be of Editable Poly instead of Cylinder


## scenes/Beechwood_0_int
  ### Articulated object (.*) instances have different scales for base links. This may have broken things during the match links script.: 7
    Articulated object qohxjq instances have different scales for base links. This may have broken things during the match links script.
    Articulated object spojpj instances have different scales for base links. This may have broken things during the match links script.
    Articulated object dmwxyl instances have different scales for base links. This may have broken things during the match links script.
    Articulated object lvgliq instances have different scales for base links. This may have broken things during the match links script.
    Articulated object ohagsq instances have different scales for base links. This may have broken things during the match links script.
    Articulated object ulnafj instances have different scales for base links. This may have broken things during the match links script.
    Articulated object immwzb instances have different scales for base links. This may have broken things during the match links script.


## scenes/Beechwood_1_int
  ### Articulated object (.*) instances have different scales for base links. This may have broken things during the match links script.: 9
    Articulated object ithrgo instances have different scales for base links. This may have broken things during the match links script.
    Articulated object lvgliq instances have different scales for base links. This may have broken things during the match links script.
    Articulated object ohagsq instances have different scales for base links. This may have broken things during the match links script.
    Articulated object dmwxyl instances have different scales for base links. This may have broken things during the match links script.
    Articulated object jhymlr instances have different scales for base links. This may have broken things during the match links script.
    Articulated object ulnafj instances have different scales for base links. This may have broken things during the match links script.
    Articulated object kubcdk instances have different scales for base links. This may have broken things during the match links script.
    Articulated object kfmkbm instances have different scales for base links. This may have broken things during the match links script.
    Articulated object pluwfl instances have different scales for base links. This may have broken things during the match links script.


## scenes/Benevolence_0_int
  ### Articulated object (.*) instances have different scales for base links. This may have broken things during the match links script.: 1
    Articulated object lvgliq instances have different scales for base links. This may have broken things during the match links script.


## scenes/Benevolence_1_int
  ### Articulated object (.*) instances have different scales for base links. This may have broken things during the match links script.: 6
    Articulated object lsyzkh instances have different scales for base links. This may have broken things during the match links script.
    Articulated object eobsmt instances have different scales for base links. This may have broken things during the match links script.
    Articulated object ufhpbn instances have different scales for base links. This may have broken things during the match links script.
    Articulated object fwstpx instances have different scales for base links. This may have broken things during the match links script.
    Articulated object ulnafj instances have different scales for base links. This may have broken things during the match links script.
    Articulated object dmwxyl instances have different scales for base links. This may have broken things during the match links script.


## scenes/Benevolence_2_int
  ### Articulated object (.*) instances have different scales for base links. This may have broken things during the match links script.: 2
    Articulated object ithrgo instances have different scales for base links. This may have broken things during the match links script.
    Articulated object lvgliq instances have different scales for base links. This may have broken things during the match links script.


## scenes/commercial_kitchen_fire_extinguisher
  ### (.*) is missing meta link: fillable.: 2
    ceaeqf is missing meta link: fillable.
    dvnwfa is missing meta link: fillable.

  ### Model ID (.*) requires joints but has no joints.: 1
    Model ID fkfgvp requires joints but has no joints.

  ### (.*) has no collision mesh. Create a collision mesh.: 1
    C-lid-wtjthb-0 has no collision mesh. Create a collision mesh.


## scenes/commercial_kitchen_pans
  ### Model ID (.*) requires joints but has no joints.: 1
    Model ID ouzyvo requires joints but has no joints.


## scenes/gates_bedroom
  ### Model ID (.*) requires joints but has no joints.: 1
    Model ID xdxisn requires joints but has no joints.


## scenes/grocery_store_asian
  ### (.*) is missing meta link: fillable.: 3
    ekvdjq is missing meta link: fillable.
    vhnlno is missing meta link: fillable.
    akinbg is missing meta link: fillable.

  ### Model ID (.*) requires joints but has no joints.: 1
    Model ID eyegld requires joints but has no joints.

  ### Articulated object (.*) instances have different scales for base links. This may have broken things during the match links script.: 1
    Articulated object xiajyb instances have different scales for base links. This may have broken things during the match links script.


## scenes/grocery_store_cafe
  ### (.*) is missing meta link: fillable.: 9
    sjmdri is missing meta link: fillable.
    ldvnrg is missing meta link: fillable.
    tfjzen is missing meta link: fillable.
    iruewm is missing meta link: fillable.
    cjhskr is missing meta link: fillable.
    dylnro is missing meta link: fillable.
    tggwud is missing meta link: fillable.
    wranvj is missing meta link: fillable.
    xngcbz is missing meta link: fillable.

  ### Model ID (.*) requires joints but has no joints.: 2
    Model ID mefezc requires joints but has no joints.
    Model ID dyiufq requires joints but has no joints.

  ### Convex mesh fillable meta link (.*) should be of Editable Poly instead of .*: 1
    Convex mesh fillable meta link F-grocery_shelf-rowulr-0-Mfillable should be of Editable Poly instead of Box

  ### fillable_seed is an invalid meta link under (.*).: 1
    fillable_seed is an invalid meta link under F-grocery_shelf-sjmdri-0.

  ### Articulated object (.*) instances have different scales for base links. This may have broken things during the match links script.: 1
    Articulated object rxvopf instances have different scales for base links. This may have broken things during the match links script.


## scenes/grocery_store_convenience
  ### Model ID (.*) requires joints but has no joints.: 2
    Model ID vzwcql requires joints but has no joints.
    Model ID xjyvwx requires joints but has no joints.

  ### (.*) is missing meta link: fillable.: 1
    bqpanz is missing meta link: fillable.


## scenes/grocery_store_half_stocked
  ### Model ID (.*) requires joints but has no joints.: 2
    Model ID crlhmi requires joints but has no joints.
    Model ID rkqcfx requires joints but has no joints.

  ### (.*) is missing meta link: fillable.: 1
    ztrupq is missing meta link: fillable.

  ### Convex mesh fillable meta link (.*) should be of Editable Poly instead of .*: 1
    Convex mesh fillable meta link F-grocery_shelf-rywinu-0-Mfillable should be of Editable Poly instead of Box


## scenes/hall_arch_wood
  ### (.*) link .* has different .* in instance .* compared to instance .*. .* difference: .*.: 11
    ijnwlp link glass has different position in instance 4 compared to instance 0. Position difference: [ 1.56390977e+00  1.97361946e+00 -9.54524148e-06].
    ijnwlp link glass has different position in instance 5 compared to instance 0. Position difference: [ 1.56392503e+00  1.97361898e+00 -9.54524148e-06].
    ijnwlp link glass has different position in instance 6 compared to instance 0. Position difference: [1.35134482e+00 1.70353699e+00 2.94575002e-05].
    ijnwlp link glass has different position in instance 3 compared to instance 0. Position difference: [ 1.56395555e+00  1.97361851e+00 -9.54535790e-06].
    ijnwlp link glass has different position in instance 2 compared to instance 0. Position difference: [ 1.56395173e+00  1.97361851e+00 -9.54535790e-06].
    ijnwlp link glass has different position in instance 8 compared to instance 0. Position difference: [1.35126853e+00 1.70353842e+00 2.94577330e-05].
    ijnwlp link glass has different position in instance 7 compared to instance 0. Position difference: [1.35134482e+00 1.70353699e+00 2.94575002e-05].
    ijnwlp link glass has different position in instance 1 compared to instance 0. Position difference: [-8.55597019e-01 -1.06878829e+00  2.97740335e-05].
    ijnwlp link glass has different position in instance 11 compared to instance 0. Position difference: [-3.73193169e+00 -4.60644865e+00  2.36311462e-06].
    ijnwlp link glass has different position in instance 9 compared to instance 0. Position difference: [-3.73193216e+00 -4.60647917e+00  2.36299820e-06].
    ijnwlp link glass has different position in instance 10 compared to instance 0. Position difference: [-3.73193288e+00 -4.60654020e+00  2.36276537e-06].

  ### Model ID (.*) requires joints but has no joints.: 3
    Model ID yxclci requires joints but has no joints.
    Model ID ijnwlp requires joints but has no joints.
    Model ID kmvdjo requires joints but has no joints.


## scenes/hall_glass_ceiling
  ### Model ID (.*) requires joints but has no joints.: 1
    Model ID cggyho requires joints but has no joints.


## scenes/hall_train_station
  ### Model ID (.*) requires joints but has no joints.: 4
    Model ID asrqdf requires joints but has no joints.
    Model ID vsifsu requires joints but has no joints.
    Model ID vqbgzc requires joints but has no joints.
    Model ID cmuvzz requires joints but has no joints.


## scenes/hotel_gym_spa
  ### (.*) link .* has different .* in instance .* compared to instance .*. .* difference: .*.: 7
    dladgw link glass has different position in instance 1 compared to instance 0. Position difference: [-78.85980225  -0.88166004  68.59760284].
    ceddpg link link_2 has different rotation in instance 1 compared to instance 0. Rotation difference: 0.025452430695023297.
    nfrbch link link_2 has different rotation in instance 1 compared to instance 0. Rotation difference: 0.10949835717174454.
    dladgw link glass has different position in instance 2 compared to instance 0. Position difference: [-7.88129272e+01  1.75926834e-04  6.85983429e+01].
    ceddpg link link_1 has different rotation in instance 1 compared to instance 0. Rotation difference: 0.025452430695023297.
    nfrbch link link_2 has different scale in instance 1 compared to instance 0. Scale difference: [1.21380937 0.8238526  1.        ].
    lcrefl link leaf has different position in instance 1 compared to instance 0. Position difference: [-0.41162109  1.10595703  0.        ].

  ### Articulated object (.*) instances have different scales for base links. This may have broken things during the match links script.: 3
    Articulated object nfrbch instances have different scales for base links. This may have broken things during the match links script.
    Articulated object dladgw instances have different scales for base links. This may have broken things during the match links script.
    Articulated object ceddpg instances have different scales for base links. This may have broken things during the match links script.

  ### (.*) has different shear. Match scaling axes on each instance.: 2
    B-door-nfrbch-1-link_2-link_1-R-lower has different shear. Match scaling axes on each instance.
    B-door-nfrbch-1-link_1-base_link-R-lower has different shear. Match scaling axes on each instance.

  ### (.*) is missing meta link: fillable.: 1
    apiaid is missing meta link: fillable.


## scenes/hotel_suite_large
  ### (.*) is missing meta link: fillable.: 4
    cbidyu is missing meta link: fillable.
    aqtmhm is missing meta link: fillable.
    rglkkf is missing meta link: fillable.
    nbmidu is missing meta link: fillable.

  ### Model ID (.*) requires joints but has no joints.: 1
    Model ID ucejof requires joints but has no joints.


## scenes/hotel_suite_small
  ### Model ID (.*) requires joints but has no joints.: 1
    Model ID vuevwo requires joints but has no joints.

  ### (.*) is missing meta link: fillable.: 1
    kxvbui is missing meta link: fillable.


## scenes/house_double_floor_lower
  ### (.*) has different UV unwrapping than recorded. Reunwrap the object.: 12
    top_cabinet-tynnnw-0-leaf3-base_link-R-upper has different UV unwrapping than recorded. Reunwrap the object.
    C-laptop-pczjnr-0 has different UV unwrapping than recorded. Reunwrap the object.
    top_cabinet-tynnnw-0-leaf0-base_link-R-lower has different UV unwrapping than recorded. Reunwrap the object.
    C-laptop-pczjnr-0-screen-base_link-R-lower has different UV unwrapping than recorded. Reunwrap the object.
    top_cabinet-tynnnw-0-leaf0-base_link-R-upper has different UV unwrapping than recorded. Reunwrap the object.
    top_cabinet-tynnnw-0-leaf2-base_link-R-upper has different UV unwrapping than recorded. Reunwrap the object.
    top_cabinet-tynnnw-0 has different UV unwrapping than recorded. Reunwrap the object.
    C-laptop-pczjnr-0-screen-base_link-R-upper has different UV unwrapping than recorded. Reunwrap the object.
    top_cabinet-tynnnw-0-leaf2-base_link-R-lower has different UV unwrapping than recorded. Reunwrap the object.
    top_cabinet-tynnnw-0-leaf1-base_link-R-lower has different UV unwrapping than recorded. Reunwrap the object.
    top_cabinet-tynnnw-0-leaf3-base_link-R-lower has different UV unwrapping than recorded. Reunwrap the object.
    top_cabinet-tynnnw-0-leaf1-base_link-R-upper has different UV unwrapping than recorded. Reunwrap the object.

  ### Model ID (.*) requires joints but has no joints.: 5
    Model ID ifgcmr requires joints but has no joints.
    Model ID byvbuc requires joints but has no joints.
    Model ID xnjqix requires joints but has no joints.
    Model ID lhzwtz requires joints but has no joints.
    Model ID kvdqep requires joints but has no joints.

  ### (.*) has no collision mesh. Create a collision mesh.: 5
    top_cabinet-tynnnw-0-leaf3-base_link-R-lower has no collision mesh. Create a collision mesh.
    C-laptop-pczjnr-0-screen-base_link-R-lower has no collision mesh. Create a collision mesh.
    top_cabinet-tynnnw-0-leaf1-base_link-R-lower has no collision mesh. Create a collision mesh.
    top_cabinet-tynnnw-0-leaf0-base_link-R-lower has no collision mesh. Create a collision mesh.
    top_cabinet-tynnnw-0-leaf2-base_link-R-lower has no collision mesh. Create a collision mesh.

  ### (.*) has different pivot offset position \(by (.*)\). Match pivots on each instance.: 5
    top_cabinet-tynnnw-0-leaf3-base_link-R-upper has different pivot offset position (by [-1.38037151e+02 -6.10351744e-05 -4.41731201e+02]). Match pivots on each instance.
    C-laptop-pczjnr-0-screen-base_link-R-upper has different pivot offset position (by [112.61002067   9.97821402  -4.03209019]). Match pivots on each instance.
    top_cabinet-tynnnw-0-leaf2-base_link-R-upper has different pivot offset position (by [-1.38037151e+02 -6.10351744e-05 -4.41731201e+02]). Match pivots on each instance.
    top_cabinet-tynnnw-0-leaf1-base_link-R-upper has different pivot offset position (by [-1.38037151e+02 -6.10351744e-05 -4.41731201e+02]). Match pivots on each instance.
    top_cabinet-tynnnw-0-leaf0-base_link-R-upper has different pivot offset position (by [-1.38037151e+02 -6.10351744e-05 -4.41731201e+02]). Match pivots on each instance.

  ### (.*) is missing meta link: fillable.: 2
    vanltx is missing meta link: fillable.
    byvbuc is missing meta link: fillable.

  ### (.*) is missing meta link: heatsource.: 1
    sxfjac is missing meta link: heatsource.


## scenes/house_double_floor_upper
  ### (.*) has different UV unwrapping than recorded. Reunwrap the object.: 10
    F-L-bottom_cabinet-ncgmqr-0-drawer0-base_link-P-lower has different UV unwrapping than recorded. Reunwrap the object.
    F-L-bottom_cabinet-ncgmqr-0-drawer0-base_link-P-upper has different UV unwrapping than recorded. Reunwrap the object.
    F-L-bottom_cabinet-ncgmqr-0-drawer2-base_link-P-upper has different UV unwrapping than recorded. Reunwrap the object.
    C-laptop-ongghk-0-screen-base_link-R-lower has different UV unwrapping than recorded. Reunwrap the object.
    C-laptop-ongghk-0-screen-base_link-R-upper has different UV unwrapping than recorded. Reunwrap the object.
    F-L-bottom_cabinet-ncgmqr-0-drawer1-base_link-P-lower has different UV unwrapping than recorded. Reunwrap the object.
    F-L-bottom_cabinet-ncgmqr-0-drawer1-base_link-P-upper has different UV unwrapping than recorded. Reunwrap the object.
    C-laptop-ongghk-0 has different UV unwrapping than recorded. Reunwrap the object.
    F-L-bottom_cabinet-ncgmqr-0 has different UV unwrapping than recorded. Reunwrap the object.
    F-L-bottom_cabinet-ncgmqr-0-drawer2-base_link-P-lower has different UV unwrapping than recorded. Reunwrap the object.

  ### (.*) link .* has different .* in instance .* compared to instance .*. .* difference: .*.: 6
    pxhbim link leaf has different rotation in instance 2 compared to instance 0. Rotation difference: 0.020404063099217943.
    pxhbim link leaf has different position in instance 3 compared to instance 0. Position difference: [ 4.07201195e+00 -2.62583561e+01 -2.43722840e-04].
    pxhbim link leaf has different rotation in instance 1 compared to instance 0. Rotation difference: 0.06251014348568772.
    pxhbim link leaf has different position in instance 2 compared to instance 0. Position difference: [ 1.92597508e+00 -8.56097889e+00 -4.88278893e-04].
    pxhbim link leaf has different position in instance 1 compared to instance 0. Position difference: [ 4.07207298e+00 -2.62583580e+01 -2.43722854e-04].
    pxhbim link leaf has different rotation in instance 3 compared to instance 0. Rotation difference: 0.06251014348568772.

  ### (.*) has no collision mesh. Create a collision mesh.: 4
    F-L-bottom_cabinet-ncgmqr-0-drawer1-base_link-P-lower has no collision mesh. Create a collision mesh.
    C-laptop-ongghk-0-screen-base_link-R-lower has no collision mesh. Create a collision mesh.
    F-L-bottom_cabinet-ncgmqr-0-drawer2-base_link-P-lower has no collision mesh. Create a collision mesh.
    F-L-bottom_cabinet-ncgmqr-0-drawer0-base_link-P-lower has no collision mesh. Create a collision mesh.

  ### Model ID (.*) requires joints but has no joints.: 1
    Model ID etjpqr requires joints but has no joints.

  ### (.*) is missing meta link: fillable.: 1
    sapyzl is missing meta link: fillable.

  ### (.*) has different pivot offset position \(by (.*)\). Match pivots on each instance.: 1
    C-laptop-ongghk-0-screen-base_link-R-upper has different pivot offset position (by [ 1.30769326e+02  1.21014912e-02 -2.33537277e+00]). Match pivots on each instance.


## scenes/house_single_floor
  ### (.*) is missing meta link: fillable.: 31
    czcujg is missing meta link: fillable.
    jtmpsj is missing meta link: fillable.
    ovipql is missing meta link: fillable.
    wauyns is missing meta link: fillable.
    cqoesa is missing meta link: fillable.
    zlaser is missing meta link: fillable.
    aviadj is missing meta link: fillable.
    upxnpu is missing meta link: fillable.
    ybzffr is missing meta link: fillable.
    amutun is missing meta link: fillable.
    todoac is missing meta link: fillable.
    vztxva is missing meta link: fillable.
    itlkfz is missing meta link: fillable.
    stibna is missing meta link: fillable.
    vxobmt is missing meta link: fillable.
    dywzhj is missing meta link: fillable.
    pfusrd is missing meta link: fillable.
    fjozhc is missing meta link: fillable.
    rrmzvc is missing meta link: fillable.
    xkwxtj is missing meta link: fillable.
    exvfii is missing meta link: fillable.
    onezuj is missing meta link: fillable.
    xipfrv is missing meta link: fillable.
    rfegnv is missing meta link: fillable.
    nkzulg is missing meta link: fillable.
    qjbbbb is missing meta link: fillable.
    otswdv is missing meta link: fillable.
    pmhcse is missing meta link: fillable.
    gvygxu is missing meta link: fillable.
    kfiujj is missing meta link: fillable.
    gwqitr is missing meta link: fillable.

  ### Model ID (.*) requires joints but has no joints.: 30
    Model ID tojcni requires joints but has no joints.
    Model ID yjbico requires joints but has no joints.
    Model ID tspbac requires joints but has no joints.
    Model ID yikxbf requires joints but has no joints.
    Model ID bvqijp requires joints but has no joints.
    Model ID jrnfkt requires joints but has no joints.
    Model ID ociqav requires joints but has no joints.
    Model ID jdwvyt requires joints but has no joints.
    Model ID usynui requires joints but has no joints.
    Model ID vxqpnm requires joints but has no joints.
    Model ID ovzepp requires joints but has no joints.
    Model ID xucvcj requires joints but has no joints.
    Model ID engzvq requires joints but has no joints.
    Model ID nszybf requires joints but has no joints.
    Model ID ijpvgf requires joints but has no joints.
    Model ID xkxvpp requires joints but has no joints.
    Model ID dfdweq requires joints but has no joints.
    Model ID ropopf requires joints but has no joints.
    Model ID yzxhbp requires joints but has no joints.
    Model ID qvqecb requires joints but has no joints.
    Model ID iggzsd requires joints but has no joints.
    Model ID glimdy requires joints but has no joints.
    Model ID dbqbzg requires joints but has no joints.
    Model ID hohdkt requires joints but has no joints.
    Model ID eyvzfu requires joints but has no joints.
    Model ID qfifse requires joints but has no joints.
    Model ID pjaljg requires joints but has no joints.
    Model ID hwunjt requires joints but has no joints.
    Model ID wogynv requires joints but has no joints.
    Model ID nruots requires joints but has no joints.

  ### (.*) has different UV unwrapping than recorded. Reunwrap the object.: 6
    C-laptop-vhbsgv-0-screen-base_link-R-upper has different UV unwrapping than recorded. Reunwrap the object.
    C-laptop-vhbsgv-0-screen-base_link-R-lower has different UV unwrapping than recorded. Reunwrap the object.
    C-laptop-izydvb-0-screen-base_link-R-upper has different UV unwrapping than recorded. Reunwrap the object.
    C-laptop-izydvb-0-screen-base_link-R-lower has different UV unwrapping than recorded. Reunwrap the object.
    C-laptop-izydvb-0 has different UV unwrapping than recorded. Reunwrap the object.
    C-laptop-vhbsgv-0 has different UV unwrapping than recorded. Reunwrap the object.

  ### Model ID (.*) contains 'todo'.: 5
    Model ID todoad contains 'todo'.
    Model ID todopi contains 'todo'.
    Model ID todoac contains 'todo'.
    Model ID todoph contains 'todo'.
    Model ID todopf contains 'todo'.

  ### (.*) link .* has different .* in instance .* compared to instance .*. .* difference: .*.: 5
    kwbnhy link glass has different position in instance 2 compared to instance 0. Position difference: [-0.0032959  -1.87890649 -1.94921863].
    kwbnhy link glass has different position in instance 3 compared to instance 0. Position difference: [-4.51660808e-03  5.65932693e+01  5.87460899e+01].
    kwbnhy link leaf has different position in instance 3 compared to instance 0. Position difference: [ -0.06717402 -53.97953033 -56.25388336].
    kwbnhy link leaf has different position in instance 2 compared to instance 0. Position difference: [-1.60175562e-03 -2.26816578e+01 -2.36718655e+01].
    kwbnhy link leaf has different position in instance 1 compared to instance 0. Position difference: [-4.53650951e-04  5.31035538e+01  5.49648209e+01].

  ### (.*) has no collision mesh. Create a collision mesh.: 2
    C-laptop-izydvb-0-screen-base_link-R-lower has no collision mesh. Create a collision mesh.
    C-laptop-vhbsgv-0-screen-base_link-R-lower has no collision mesh. Create a collision mesh.

  ### (.*) has different pivot offset position \(by (.*)\). Match pivots on each instance.: 2
    C-laptop-vhbsgv-0-screen-base_link-R-upper has different pivot offset position (by [122.07003021   0.31538305 -13.12661648]). Match pivots on each instance.
    C-laptop-izydvb-0-screen-base_link-R-upper has different pivot offset position (by [1.23282590e+02 0.00000000e+00 5.38885357e-06]). Match pivots on each instance.

  ### Articulated object (.*) instances have different scales for base links. This may have broken things during the match links script.: 1
    Articulated object gjrero instances have different scales for base links. This may have broken things during the match links script.


## scenes/Ihlen_0_int
  ### Articulated object (.*) instances have different scales for base links. This may have broken things during the match links script.: 3
    Articulated object ulnafj instances have different scales for base links. This may have broken things during the match links script.
    Articulated object ktydvs instances have different scales for base links. This may have broken things during the match links script.
    Articulated object lvgliq instances have different scales for base links. This may have broken things during the match links script.


## scenes/Ihlen_1_int
  ### Articulated object (.*) instances have different scales for base links. This may have broken things during the match links script.: 8
    Articulated object qohxjq instances have different scales for base links. This may have broken things during the match links script.
    Articulated object ulnafj instances have different scales for base links. This may have broken things during the match links script.
    Articulated object lvgliq instances have different scales for base links. This may have broken things during the match links script.
    Articulated object jrhgeu instances have different scales for base links. This may have broken things during the match links script.
    Articulated object xbfgjc instances have different scales for base links. This may have broken things during the match links script.
    Articulated object ithrgo instances have different scales for base links. This may have broken things during the match links script.
    Articulated object fwstpx instances have different scales for base links. This may have broken things during the match links script.
    Articulated object dmwxyl instances have different scales for base links. This may have broken things during the match links script.


## scenes/Merom_0_garden
  ### Articulated object (.*) instances have different scales for base links. This may have broken things during the match links script.: 2
    Articulated object ulnafj instances have different scales for base links. This may have broken things during the match links script.
    Articulated object lvgliq instances have different scales for base links. This may have broken things during the match links script.

  ### Model ID (.*) requires joints but has no joints.: 1
    Model ID surlxy requires joints but has no joints.


## scenes/Merom_0_int
  ### Articulated object (.*) instances have different scales for base links. This may have broken things during the match links script.: 2
    Articulated object ulnafj instances have different scales for base links. This may have broken things during the match links script.
    Articulated object lvgliq instances have different scales for base links. This may have broken things during the match links script.


## scenes/Merom_1_int
  ### Articulated object (.*) instances have different scales for base links. This may have broken things during the match links script.: 7
    Articulated object lvgliq instances have different scales for base links. This may have broken things during the match links script.
    Articulated object slgzfc instances have different scales for base links. This may have broken things during the match links script.
    Articulated object xbfgjc instances have different scales for base links. This may have broken things during the match links script.
    Articulated object dmwxyl instances have different scales for base links. This may have broken things during the match links script.
    Articulated object ithrgo instances have different scales for base links. This may have broken things during the match links script.
    Articulated object ohagsq instances have different scales for base links. This may have broken things during the match links script.
    Articulated object ulnafj instances have different scales for base links. This may have broken things during the match links script.


## scenes/office_bike
  ### Model ID (.*) requires joints but has no joints.: 2
    Model ID fqsuee requires joints but has no joints.
    Model ID mvutvn requires joints but has no joints.


## scenes/office_cubicles_left
  ### (.*) has different pivot offset position \(by (.*)\). Match pivots on each instance.: 5
    B-conference_table-qzmjrj-3-base_link has different pivot offset position (by [1.99861034e+02 1.89270300e+03 3.88318927e-06]). Match pivots on each instance.
    B-conference_table-qzmjrj-5-base_link has different pivot offset position (by [ 1.99865835e+02  1.89270828e+03 -1.74012047e-05]). Match pivots on each instance.
    B-conference_table-qzmjrj-4-base_link has different pivot offset position (by [ 1.99861053e+02  1.89270300e+03 -1.41343231e-05]). Match pivots on each instance.
    B-conference_table-qzmjrj-2-base_link has different pivot offset position (by [ 1.99861608e+02  1.89270363e+03 -1.42983756e-05]). Match pivots on each instance.
    B-conference_table-qzmjrj-1-base_link has different pivot offset position (by [ 1.96635841e+02  1.88910358e+03 -1.43884486e-05]). Match pivots on each instance.

  ### Articulated object (.*) instances have different scales for base links. This may have broken things during the match links script.: 1
    Articulated object immwzb instances have different scales for base links. This may have broken things during the match links script.


## scenes/office_cubicles_right
  ### Model ID (.*) requires joints but has no joints.: 1
    Model ID yazaic requires joints but has no joints.


## scenes/office_large
  ### (.*) has scale that is not 1. Reset scale.: 57
    B-C-mouse-chqwqp-20-base_link has scale that is not 1. Reset scale.
    B-C-mouse-chqwqp-28-base_link has scale that is not 1. Reset scale.
    B-C-mouse-chqwqp-53-base_link has scale that is not 1. Reset scale.
    B-C-mouse-chqwqp-19-base_link has scale that is not 1. Reset scale.
    B-C-mouse-chqwqp-27-base_link has scale that is not 1. Reset scale.
    B-C-mouse-chqwqp-31-base_link has scale that is not 1. Reset scale.
    B-C-mouse-chqwqp-34-base_link has scale that is not 1. Reset scale.
    B-C-mouse-chqwqp-2-base_link has scale that is not 1. Reset scale.
    B-C-mouse-chqwqp-0-base_link has scale that is not 1. Reset scale.
    B-C-mouse-chqwqp-7-base_link has scale that is not 1. Reset scale.
    B-C-mouse-chqwqp-54-base_link has scale that is not 1. Reset scale.
    B-C-mouse-chqwqp-18-base_link has scale that is not 1. Reset scale.
    B-C-mouse-chqwqp-26-base_link has scale that is not 1. Reset scale.
    B-C-mouse-chqwqp-36-base_link has scale that is not 1. Reset scale.
    B-C-mouse-chqwqp-42-base_link has scale that is not 1. Reset scale.
    B-C-mouse-chqwqp-3-base_link has scale that is not 1. Reset scale.
    B-C-mouse-chqwqp-9-base_link has scale that is not 1. Reset scale.
    B-C-mouse-chqwqp-39-base_link has scale that is not 1. Reset scale.
    B-C-mouse-chqwqp-51-base_link has scale that is not 1. Reset scale.
    B-C-mouse-chqwqp-52-base_link has scale that is not 1. Reset scale.
    B-C-mouse-chqwqp-17-base_link has scale that is not 1. Reset scale.
    B-C-mouse-chqwqp-33-base_link has scale that is not 1. Reset scale.
    B-C-mouse-chqwqp-1-base_link has scale that is not 1. Reset scale.
    B-C-mouse-chqwqp-48-base_link has scale that is not 1. Reset scale.
    B-C-mouse-chqwqp-16-base_link has scale that is not 1. Reset scale.
    B-C-mouse-chqwqp-24-base_link has scale that is not 1. Reset scale.
    B-C-mouse-chqwqp-29-base_link has scale that is not 1. Reset scale.
    B-C-mouse-chqwqp-41-base_link has scale that is not 1. Reset scale.
    B-C-mouse-chqwqp-4-base_link has scale that is not 1. Reset scale.
    B-C-mouse-chqwqp-8-base_link has scale that is not 1. Reset scale.
    B-C-mouse-chqwqp-35-base_link has scale that is not 1. Reset scale.
    B-C-mouse-chqwqp-50-base_link has scale that is not 1. Reset scale.
    B-C-mouse-chqwqp-14-base_link has scale that is not 1. Reset scale.
    B-C-mouse-chqwqp-15-base_link has scale that is not 1. Reset scale.
    B-C-mouse-chqwqp-32-base_link has scale that is not 1. Reset scale.
    B-C-mouse-chqwqp-44-base_link has scale that is not 1. Reset scale.
    B-C-mouse-chqwqp-13-base_link has scale that is not 1. Reset scale.
    B-C-mouse-chqwqp-23-base_link has scale that is not 1. Reset scale.
    B-C-mouse-chqwqp-30-base_link has scale that is not 1. Reset scale.
    B-C-mouse-chqwqp-55-base_link has scale that is not 1. Reset scale.
    B-C-mouse-chqwqp-5-base_link has scale that is not 1. Reset scale.
    B-C-mouse-chqwqp-12-base_link has scale that is not 1. Reset scale.
    B-C-mouse-chqwqp-43-base_link has scale that is not 1. Reset scale.
    B-C-mouse-chqwqp-45-base_link has scale that is not 1. Reset scale.
    B-C-mouse-chqwqp-11-base_link has scale that is not 1. Reset scale.
    B-C-mouse-chqwqp-22-base_link has scale that is not 1. Reset scale.
    B-C-mouse-chqwqp-38-base_link has scale that is not 1. Reset scale.
    B-C-mouse-chqwqp-40-base_link has scale that is not 1. Reset scale.
    B-C-mouse-chqwqp-21-base_link has scale that is not 1. Reset scale.
    B-C-mouse-chqwqp-37-base_link has scale that is not 1. Reset scale.
    B-C-mouse-chqwqp-49-base_link has scale that is not 1. Reset scale.
    B-C-mouse-chqwqp-56-base_link has scale that is not 1. Reset scale.
    B-C-mouse-chqwqp-6-base_link has scale that is not 1. Reset scale.
    B-C-mouse-chqwqp-10-base_link has scale that is not 1. Reset scale.
    B-C-mouse-chqwqp-46-base_link has scale that is not 1. Reset scale.
    B-C-mouse-chqwqp-47-base_link has scale that is not 1. Reset scale.
    B-C-mouse-chqwqp-25-base_link has scale that is not 1. Reset scale.

  ### (.*) is in the deletion queue. Delete the object.: 8
    C-cup-vlikrk-1 is in the deletion queue. Delete the object.
    C-cup-vlikrk-4 is in the deletion queue. Delete the object.
    C-cup-vlikrk-2 is in the deletion queue. Delete the object.
    C-cup-vlikrk-7 is in the deletion queue. Delete the object.
    C-cup-vlikrk-5 is in the deletion queue. Delete the object.
    C-cup-vlikrk-3 is in the deletion queue. Delete the object.
    C-cup-vlikrk-6 is in the deletion queue. Delete the object.
    C-cup-vlikrk-0 is in the deletion queue. Delete the object.

  ### Articulated object (.*) instances have different scales for base links. This may have broken things during the match links script.: 4
    Articulated object rvunhj instances have different scales for base links. This may have broken things during the match links script.
    Articulated object jhymlr instances have different scales for base links. This may have broken things during the match links script.
    Articulated object bamfsz instances have different scales for base links. This may have broken things during the match links script.
    Articulated object zuwvdo instances have different scales for base links. This may have broken things during the match links script.

  ### (.*) has unapplied rename .*.: 3
    B-C-cup-nhzrei-2-base_link has unapplied rename cup.
    B-C-cup-nhzrei-0-base_link has unapplied rename cup.
    B-C-cup-nhzrei-1-base_link has unapplied rename cup.

  ### Model ID (.*) requires joints but has no joints.: 2
    Model ID ymwnju requires joints but has no joints.
    Model ID zlmfwf requires joints but has no joints.

  ### (.*) link .* has different .* in instance .* compared to instance .*. .* difference: .*.: 2
    dqnbsj link leaf has different position in instance 1 compared to instance 0. Position difference: [8.30593288e-01 3.08849849e-07 1.73283505e+00].
    dqnbsj link leaf has different rotation in instance 1 compared to instance 0. Rotation difference: 0.020089581174912937.


## scenes/office_vendor_machine
  ### Model ID (.*) requires joints but has no joints.: 1
    Model ID ygrams requires joints but has no joints.

  ### (.*) is missing meta link: fillable.: 1
    ygrams is missing meta link: fillable.


## scenes/Pomaria_0_garden
  ### (.*) is missing meta link: fillable.: 3
    supuws is missing meta link: fillable.
    qgfbdj is missing meta link: fillable.
    xezdng is missing meta link: fillable.

  ### Model ID (.*) requires joints but has no joints.: 2
    Model ID supuws requires joints but has no joints.
    Model ID ysgkig requires joints but has no joints.

  ### Articulated object (.*) instances have different scales for base links. This may have broken things during the match links script.: 2
    Articulated object lvgliq instances have different scales for base links. This may have broken things during the match links script.
    Articulated object spojpj instances have different scales for base links. This may have broken things during the match links script.


## scenes/Pomaria_0_int
  ### Articulated object (.*) instances have different scales for base links. This may have broken things during the match links script.: 4
    Articulated object ohagsq instances have different scales for base links. This may have broken things during the match links script.
    Articulated object spojpj instances have different scales for base links. This may have broken things during the match links script.
    Articulated object lvgliq instances have different scales for base links. This may have broken things during the match links script.
    Articulated object ulnafj instances have different scales for base links. This may have broken things during the match links script.


## scenes/Pomaria_1_int
  ### Articulated object (.*) instances have different scales for base links. This may have broken things during the match links script.: 11
    Articulated object lsyzkh instances have different scales for base links. This may have broken things during the match links script.
    Articulated object ulnafj instances have different scales for base links. This may have broken things during the match links script.
    Articulated object xyejdx instances have different scales for base links. This may have broken things during the match links script.
    Articulated object ithrgo instances have different scales for base links. This may have broken things during the match links script.
    Articulated object qohxjq instances have different scales for base links. This may have broken things during the match links script.
    Articulated object spojpj instances have different scales for base links. This may have broken things during the match links script.
    Articulated object fexqbj instances have different scales for base links. This may have broken things during the match links script.
    Articulated object bmsclc instances have different scales for base links. This may have broken things during the match links script.
    Articulated object dmwxyl instances have different scales for base links. This may have broken things during the match links script.
    Articulated object lvgliq instances have different scales for base links. This may have broken things during the match links script.
    Articulated object ohagsq instances have different scales for base links. This may have broken things during the match links script.


## scenes/Pomaria_2_int
  ### Articulated object (.*) instances have different scales for base links. This may have broken things during the match links script.: 4
    Articulated object qohxjq instances have different scales for base links. This may have broken things during the match links script.
    Articulated object ohagsq instances have different scales for base links. This may have broken things during the match links script.
    Articulated object ithrgo instances have different scales for base links. This may have broken things during the match links script.
    Articulated object lvgliq instances have different scales for base links. This may have broken things during the match links script.


## scenes/public_restroom_blue
  ### Model ID (.*) requires joints but has no joints.: 1
    Model ID wkzfdu requires joints but has no joints.


## scenes/public_restroom_brown
  ### Model ID (.*) requires joints but has no joints.: 1
    Model ID qjcodu requires joints but has no joints.


## scenes/restaurant_asian
  ### (.*) has no collision mesh. Create a collision mesh.: 1
    C-bottle_of_sake-zrkfim-0 has no collision mesh. Create a collision mesh.


## scenes/restaurant_brunch
  ### (.*) is missing meta link: fillable.: 6
    qrmiur is missing meta link: fillable.
    ibrbnl is missing meta link: fillable.
    adgrkb is missing meta link: fillable.
    hrfwyd is missing meta link: fillable.
    zadqqv is missing meta link: fillable.
    cyckht is missing meta link: fillable.

  ### Model ID (.*) requires joints but has no joints.: 3
    Model ID zdeyzf requires joints but has no joints.
    Model ID waousd requires joints but has no joints.
    Model ID ywetvt requires joints but has no joints.


## scenes/restaurant_cafeteria
  ### Model ID (.*) requires joints but has no joints.: 8
    Model ID oszzno requires joints but has no joints.
    Model ID djqiqc requires joints but has no joints.
    Model ID cgchwh requires joints but has no joints.
    Model ID cciwhk requires joints but has no joints.
    Model ID lvuvbf requires joints but has no joints.
    Model ID whujjc requires joints but has no joints.
    Model ID ayvcgc requires joints but has no joints.
    Model ID guhrnv requires joints but has no joints.


## scenes/restaurant_diner
  ### Model ID (.*) requires joints but has no joints.: 2
    Model ID caymim requires joints but has no joints.
    Model ID iueiur requires joints but has no joints.


## scenes/restaurant_hotel
  ### Model ID (.*) requires joints but has no joints.: 1
    Model ID ozlemc requires joints but has no joints.

  ### (.*) is missing meta link: fillable.: 1
    ubsonj is missing meta link: fillable.


## scenes/restaurant_urban
  ### Model ID (.*) requires joints but has no joints.: 2
    Model ID txxinx requires joints but has no joints.
    Model ID vapbnz requires joints but has no joints.

  ### (.*) is missing meta link: fillable.: 1
    tgcuwh is missing meta link: fillable.


## scenes/Rs_garden
  ### Articulated object (.*) instances have different scales for base links. This may have broken things during the match links script.: 4
    Articulated object ithrgo instances have different scales for base links. This may have broken things during the match links script.
    Articulated object slgzfc instances have different scales for base links. This may have broken things during the match links script.
    Articulated object bamfsz instances have different scales for base links. This may have broken things during the match links script.
    Articulated object lsyzkh instances have different scales for base links. This may have broken things during the match links script.

  ### (.*) is missing meta link: heatsource.: 1
    dtfail is missing meta link: heatsource.


## scenes/Rs_int
  ### Articulated object (.*) instances have different scales for base links. This may have broken things during the match links script.: 4
    Articulated object ithrgo instances have different scales for base links. This may have broken things during the match links script.
    Articulated object slgzfc instances have different scales for base links. This may have broken things during the match links script.
    Articulated object bamfsz instances have different scales for base links. This may have broken things during the match links script.
    Articulated object lsyzkh instances have different scales for base links. This may have broken things during the match links script.


## scenes/school_biology
  ### (.*) link .* has different .* in instance .* compared to instance .*. .* difference: .*.: 10
    rgmujm link glass has different position in instance 8 compared to instance 0. Position difference: [ 2.7963984  -0.35243672 -0.02929547].
    rgmujm link glass has different position in instance 4 compared to instance 0. Position difference: [ 2.85645771 -0.37098923 -0.02953958].
    rgmujm link glass has different position in instance 5 compared to instance 0. Position difference: [ 2.79102731 -0.35634318 -0.02892926].
    rgmujm link glass has different position in instance 7 compared to instance 0. Position difference: [ 2.82813668 -0.35829493 -0.02856303].
    rgmujm link glass has different position in instance 3 compared to instance 0. Position difference: [ 2.85157466 -0.37001285 -0.02929544].
    xxipyh link glass has different position in instance 2 compared to instance 0. Position difference: [-62.50593567 -16.50537109  -9.28171921].
    rgmujm link glass has different position in instance 1 compared to instance 0. Position difference: [ 2.85645771 -0.37098923 -0.02966165].
    rgmujm link glass has different position in instance 2 compared to instance 0. Position difference: [ 2.85548115 -0.37098926 -0.02892923].
    xxipyh link glass has different position in instance 1 compared to instance 0. Position difference: [-9.45314217  0.05038834 -1.33642316].
    rgmujm link glass has different position in instance 6 compared to instance 0. Position difference: [ 2.79566598 -0.35243675 -0.02868512].

  ### (.*) has different pivot offset position \(by (.*)\). Match pivots on each instance.: 10
    B-window-rgmujm-5-base_link has different pivot offset position (by [-0.02527591  2.70499098  0.39255019]). Match pivots on each instance.
    B-window-rgmujm-4-base_link has different pivot offset position (by [-0.02579961  2.76733215  0.40115534]). Match pivots on each instance.
    B-window-rgmujm-3-base_link has different pivot offset position (by [-0.02538114  2.76296268  0.40027277]). Match pivots on each instance.
    B-window-rgmujm-2-base_link has different pivot offset position (by [-0.02525208  2.76670139  0.40135531]). Match pivots on each instance.
    B-window-rgmujm-1-base_link has different pivot offset position (by [-0.02591715  2.76672269  0.40117849]). Match pivots on each instance.
    B-door-xxipyh-2-base_link has different pivot offset position (by [-9.27706683e+00  9.19020755e-04 -6.57349781e+01]). Match pivots on each instance.
    B-door-xxipyh-1-base_link has different pivot offset position (by [-1.33284417e+00  8.61478489e-04 -9.45814202e+00]). Match pivots on each instance.
    B-window-rgmujm-8-base_link has different pivot offset position (by [-0.02550704  2.71015109  0.39258857]). Match pivots on each instance.
    B-window-rgmujm-7-base_link has different pivot offset position (by [-0.02476892  2.73936624  0.39650348]). Match pivots on each instance.
    B-window-rgmujm-6-base_link has different pivot offset position (by [-0.02482154  2.70897087  0.39261173]). Match pivots on each instance.

  ### Articulated object (.*) instances have different scales for base links. This may have broken things during the match links script.: 2
    Articulated object xxipyh instances have different scales for base links. This may have broken things during the match links script.
    Articulated object rgmujm instances have different scales for base links. This may have broken things during the match links script.


## scenes/school_chemistry
  ### (.*) has different pivot offset position \(by (.*)\). Match pivots on each instance.: 8
    B-door-xxipyh-2-glass-base_link-F-lower has different pivot offset position (by [8.62983556e+01 3.12551687e-04 6.20754252e+02]). Match pivots on each instance.
    B-door-xxipyh-2-reddoor1-base_link-R-lower has different pivot offset position (by [9.30352746e-01 7.98172639e-05 2.98522606e-04]). Match pivots on each instance.
    B-door-xxipyh-1-glass-base_link-F-lower has different pivot offset position (by [-1.42018635e+00 -1.72623837e-04 -1.00812663e+01]). Match pivots on each instance.
    B-door-xxipyh-2-glass1-reddoor1-F-lower has different pivot offset position (by [9.30329659e-01 1.87821513e-05 7.14206612e-05]). Match pivots on each instance.
    B-door-xxipyh-2-base_link has different pivot offset position (by [8.62983556e+01 3.12551687e-04 6.20754252e+02]). Match pivots on each instance.
    B-door-xxipyh-1-base_link has different pivot offset position (by [-1.42018635e+00 -1.72623837e-04 -1.00812663e+01]). Match pivots on each instance.
    B-door-xxipyh-2-glass2-reddoor2-F-lower has different pivot offset position (by [ 9.29709254e-01 -3.70001436e-06  2.10018310e-03]). Match pivots on each instance.
    B-door-xxipyh-2-reddoor2-base_link-R-lower has different pivot offset position (by [ 9.29733305e-01 -3.70001436e-06  2.32670523e-03]). Match pivots on each instance.

  ### Articulated object (.*) instances have different scales for base links. This may have broken things during the match links script.: 3
    Articulated object rgmujm instances have different scales for base links. This may have broken things during the match links script.
    Articulated object xxipyh instances have different scales for base links. This may have broken things during the match links script.
    Articulated object dnyzym instances have different scales for base links. This may have broken things during the match links script.

  ### Model ID (.*) requires joints but has no joints.: 1
    Model ID uztisk requires joints but has no joints.


## scenes/school_computer_lab_and_infirmary
  ### (.*) link .* has different .* in instance .* compared to instance .*. .* difference: .*.: 33
    rgmujm link moving has different position in instance 1 compared to instance 0. Position difference: [ 2.34023438e+01 -1.22382927e+01 -4.79030743e-04].
    rgmujm link moving has different position in instance 6 compared to instance 0. Position difference: [ 2.34040527e+01 -1.22031364e+01  9.22348408e-06].
    uztisk link glass_left has different position in instance 15 compared to instance 0. Position difference: [ 1.22072116e-01 -1.59191883e+00 -7.32421875e-04].
    uztisk link glass_mid has different position in instance 11 compared to instance 0. Position difference: [ 1.23048693e-01 -1.61120594e+00 -9.76562500e-04].
    rgmujm link glass has different position in instance 4 compared to instance 0. Position difference: [-113.1337738    -1.58222628   -6.09197855].
    rgmujm link glass_moving has different position in instance 5 compared to instance 0. Position difference: [-89.72601318 -13.8145752   -6.09234476].
    uztisk link glass_mid has different position in instance 16 compared to instance 0. Position difference: [ 1.22072116e-01 -1.59204090e+00 -7.32421875e-04].
    rgmujm link moving has different position in instance 4 compared to instance 0. Position difference: [ 2.34040527e+01 -1.22031364e+01  9.22348408e-06].
    uztisk link glass_mid has different position in instance 10 compared to instance 0. Position difference: [ 1.23048693e-01 -1.61120594e+00 -7.32421875e-04].
    rgmujm link moving has different position in instance 3 compared to instance 0. Position difference: [ 2.34042969e+01 -1.22031364e+01 -3.56987497e-04].
    rgmujm link glass has different position in instance 3 compared to instance 0. Position difference: [-113.13279724   -1.58222628   -6.09222269].
    uztisk link glass_right has different position in instance 11 compared to instance 0. Position difference: [ 1.23048693e-01 -1.61120594e+00 -9.76562500e-04].
    uztisk link glass_left has different position in instance 11 compared to instance 0. Position difference: [ 1.23048693e-01 -1.61120594e+00 -9.76562500e-04].
    rgmujm link glass_moving has different position in instance 4 compared to instance 0. Position difference: [-89.72723389 -13.8145752   -6.09185648].
    rgmujm link glass_moving has different position in instance 6 compared to instance 0. Position difference: [-89.72674561 -13.81359863  -6.09197855].
    uztisk link glass_right has different position in instance 10 compared to instance 0. Position difference: [ 1.23048693e-01 -1.61120594e+00 -7.32421875e-04].
    rgmujm link glass has different position in instance 5 compared to instance 0. Position difference: [-113.13279724   -1.58222628   -6.09222269].
    uztisk link glass_left has different position in instance 12 compared to instance 0. Position difference: [ 1.16578951e-01 -1.59179676e+00 -4.88281250e-04].
    rgmujm link glass has different position in instance 2 compared to instance 0. Position difference: [-113.13279724   -1.58222628   -6.09222269].
    uztisk link glass_right has different position in instance 12 compared to instance 0. Position difference: [ 1.16578951e-01 -1.59179676e+00 -4.88281250e-04].
    rgmujm link glass has different position in instance 1 compared to instance 0. Position difference: [-113.13475037   -1.58710909   -6.09210062].
    uztisk link glass_left has different position in instance 16 compared to instance 0. Position difference: [ 1.22072116e-01 -1.59204090e+00 -7.32421875e-04].
    uztisk link glass_right has different position in instance 15 compared to instance 0. Position difference: [ 1.22072116e-01 -1.59191883e+00 -7.32421875e-04].
    rgmujm link moving has different position in instance 5 compared to instance 0. Position difference: [ 2.34062500e+01 -1.22031364e+01 -2.34917185e-04].
    rgmujm link glass_moving has different position in instance 1 compared to instance 0. Position difference: [-89.72991943 -13.85461426  -6.09246683].
    uztisk link glass_mid has different position in instance 12 compared to instance 0. Position difference: [ 1.16578951e-01 -1.59179676e+00 -4.88281250e-04].
    rgmujm link glass_moving has different position in instance 2 compared to instance 0. Position difference: [-89.72601318 -13.8145752   -6.09234476].
    rgmujm link glass has different position in instance 6 compared to instance 0. Position difference: [-113.13352966   -1.58222628   -6.09197855].
    uztisk link glass_mid has different position in instance 15 compared to instance 0. Position difference: [ 1.22072116e-01 -1.59191883e+00 -7.32421875e-04].
    uztisk link glass_left has different position in instance 10 compared to instance 0. Position difference: [ 1.23048693e-01 -1.61120594e+00 -7.32421875e-04].
    uztisk link glass_right has different position in instance 16 compared to instance 0. Position difference: [ 1.22072116e-01 -1.59204090e+00 -7.32421875e-04].
    rgmujm link glass_moving has different position in instance 3 compared to instance 0. Position difference: [-89.72796631 -13.8145752   -6.09246683].
    rgmujm link moving has different position in instance 2 compared to instance 0. Position difference: [ 2.34062500e+01 -1.22031364e+01 -3.56987526e-04].

  ### (.*) has different pivot offset position \(by (.*)\). Match pivots on each instance.: 10
    B-door-xxipyh-2-base_link has different pivot offset position (by [7.64395426e+00 5.48928405e-04 5.41976097e+01]). Match pivots on each instance.
    B-door-xxipyh-1-base_link has different pivot offset position (by [ 2.93140535e+01 -1.27505229e-04  2.08535944e+02]). Match pivots on each instance.
    B-door-xxipyh-3-glass-base_link-F-lower has different pivot offset position (by [0.35918215 0.         2.54359662]). Match pivots on each instance.
    B-door-xxipyh-2-glass-base_link-F-lower has different pivot offset position (by [7.64395426e+00 5.48928405e-04 5.41976097e+01]). Match pivots on each instance.
    B-door-xxipyh-1-glass-base_link-F-lower has different pivot offset position (by [ 2.93140535e+01 -1.27505229e-04  2.08535944e+02]). Match pivots on each instance.
    B-door-xxipyh-1-glass1-reddoor1-F-lower has different pivot offset position (by [ 3.16044346e-01 -2.53545739e-05  3.39755491e-05]). Match pivots on each instance.
    B-door-xxipyh-1-glass2-reddoor2-F-lower has different pivot offset position (by [3.15831503e-01 3.01187868e-07 7.04742988e-04]). Match pivots on each instance.
    B-door-xxipyh-1-reddoor2-base_link-R-lower has different pivot offset position (by [3.15814491e-01 4.68236104e-06 7.81183107e-04]). Match pivots on each instance.
    B-door-xxipyh-1-reddoor1-base_link-R-lower has different pivot offset position (by [ 3.16035758e-01 -2.53545739e-05  1.05557785e-04]). Match pivots on each instance.
    B-door-xxipyh-3-base_link has different pivot offset position (by [0.35918215 0.         2.54359662]). Match pivots on each instance.

  ### Articulated object (.*) instances have different scales for base links. This may have broken things during the match links script.: 4
    Articulated object xxipyh instances have different scales for base links. This may have broken things during the match links script.
    Articulated object uztisk instances have different scales for base links. This may have broken things during the match links script.
    Articulated object nwlbit instances have different scales for base links. This may have broken things during the match links script.
    Articulated object rgmujm instances have different scales for base links. This may have broken things during the match links script.

  ### (.*) is missing meta link: fillable.: 2
    lufqkq is missing meta link: fillable.
    inkwmw is missing meta link: fillable.


## scenes/school_geography
  ### (.*) has different pivot offset position \(by (.*)\). Match pivots on each instance.: 18
    B-window-rgmujm-2-glass_moving-moving-F-lower has different pivot offset position (by [0.00048436 0.18248752 0.02690914]). Match pivots on each instance.
    B-window-rgmujm-10-glass-base_link-F-lower has different pivot offset position (by [0.00137874 0.13052054 0.01954653]). Match pivots on each instance.
    B-window-rgmujm-6-glass_moving-moving-F-lower has different pivot offset position (by [0.00229153 0.20122408 0.03017341]). Match pivots on each instance.
    B-window-rgmujm-9-glass-base_link-F-lower has different pivot offset position (by [0.00141969 0.12803441 0.01917296]). Match pivots on each instance.
    B-window-rgmujm-8-glass-base_link-F-lower has different pivot offset position (by [0.00122153 0.12803441 0.01911609]). Match pivots on each instance.
    B-window-rgmujm-7-glass-base_link-F-lower has different pivot offset position (by [0.00152594 0.13028448 0.01956934]). Match pivots on each instance.
    B-window-rgmujm-6-glass-base_link-F-lower has different pivot offset position (by [0.00229562 0.20128336 0.03020768]). Match pivots on each instance.
    B-window-rgmujm-5-glass-base_link-F-lower has different pivot offset position (by [0.0021789  0.18290755 0.02747731]). Match pivots on each instance.
    B-window-rgmujm-4-glass-base_link-F-lower has different pivot offset position (by [0.00247217 0.18625443 0.02804858]). Match pivots on each instance.
    B-window-rgmujm-7-glass_moving-moving-F-lower has different pivot offset position (by [0.00156758 0.13010246 0.01954265]). Match pivots on each instance.
    B-window-rgmujm-3-glass-base_link-F-lower has different pivot offset position (by [0.00107613 0.18132858 0.02687878]). Match pivots on each instance.
    B-window-rgmujm-2-glass-base_link-F-lower has different pivot offset position (by [0.0005219  0.18242049 0.02687878]). Match pivots on each instance.
    B-window-rgmujm-10-glass_moving-moving-F-lower has different pivot offset position (by [0.0013412  0.13056635 0.01954646]). Match pivots on each instance.
    B-window-rgmujm-8-glass_moving-moving-F-lower has different pivot offset position (by [0.00112301 0.12802121 0.01910469]). Match pivots on each instance.
    B-window-rgmujm-9-glass_moving-moving-F-lower has different pivot offset position (by [0.00138625 0.12802945 0.0191919 ]). Match pivots on each instance.
    B-window-rgmujm-5-glass_moving-moving-F-lower has different pivot offset position (by [0.00207334 0.18288122 0.02743917]). Match pivots on each instance.
    B-window-rgmujm-4-glass_moving-moving-F-lower has different pivot offset position (by [0.00242349 0.18627889 0.02802186]). Match pivots on each instance.
    B-window-rgmujm-3-glass_moving-moving-F-lower has different pivot offset position (by [0.00103859 0.1813661  0.02690914]). Match pivots on each instance.

  ### (.*) link .* has different .* in instance .* compared to instance .*. .* difference: .*.: 4
    xxipyh link glass has different position in instance 2 compared to instance 0. Position difference: [-0.78515673  0.79101521 -4.546875  ].
    ehwmol link door1 has different position in instance 1 compared to instance 0. Position difference: [-1.16295500e+01 -8.74582678e-02  8.87818984e-04].
    xxipyh link glass has different position in instance 3 compared to instance 0. Position difference: [-4.78516912 22.62499809 -2.44091797].
    xxipyh link glass has different position in instance 1 compared to instance 0. Position difference: [-0.80273485  0.78808546 -4.54736328].

  ### Articulated object (.*) instances have different scales for base links. This may have broken things during the match links script.: 1
    Articulated object rgmujm instances have different scales for base links. This may have broken things during the match links script.


## scenes/school_gym
  ### (.*) has different pivot offset position \(by (.*)\). Match pivots on each instance.: 12
    B-door-xxipyh-2-glass1-reddoor1-F-lower has different pivot offset position (by [-0.11845538 -0.00069637  0.00220266]). Match pivots on each instance.
    B-door-xxipyh-1-glass1-reddoor1-F-lower has different pivot offset position (by [-0.11899916 -0.00062977 -0.00125448]). Match pivots on each instance.
    B-door-xxipyh-1-glass-base_link-F-lower has different pivot offset position (by [-1.64756043e+00 -1.83184960e-03 -1.16791741e+01]). Match pivots on each instance.
    B-door-xxipyh-3-glass2-reddoor2-F-lower has different pivot offset position (by [-0.66281528 -0.00066623 -0.00277469]). Match pivots on each instance.
    B-door-xxipyh-2-glass2-reddoor2-F-lower has different pivot offset position (by [-0.11850849 -0.00027155 -0.00057246]). Match pivots on each instance.
    B-door-xxipyh-1-glass2-reddoor2-F-lower has different pivot offset position (by [-0.11907878 -0.00038168 -0.00102756]). Match pivots on each instance.
    B-door-xxipyh-2-glass-base_link-F-lower has different pivot offset position (by [-1.63967109e+00 -2.01058356e-03 -1.16205804e+01]). Match pivots on each instance.
    B-door-xxipyh-3-glass-base_link-F-lower has different pivot offset position (by [-9.16971916e+00 -9.76562558e-04 -6.49314269e+01]). Match pivots on each instance.
    B-door-xxipyh-3-glass1-reddoor1-F-lower has different pivot offset position (by [-6.62805939e-01  4.68862594e-04  7.01439113e-04]). Match pivots on each instance.
    B-door-xxipyh-3-base_link has different pivot offset position (by [-9.16478800e+00  1.52549081e-03 -6.49431464e+01]). Match pivots on each instance.
    B-door-xxipyh-2-base_link has different pivot offset position (by [-1.63992503e+00 -4.85175524e-04 -1.16323006e+01]). Match pivots on each instance.
    B-door-xxipyh-1-base_link has different pivot offset position (by [-1.64780945e+00 -3.06340396e-04 -1.16869881e+01]). Match pivots on each instance.

  ### Model ID (.*) requires joints but has no joints.: 4
    Model ID dompiq requires joints but has no joints.
    Model ID khxpgc requires joints but has no joints.
    Model ID ocpgvi requires joints but has no joints.
    Model ID ebtgyg requires joints but has no joints.

  ### (.*) link .* has different .* in instance .* compared to instance .*. .* difference: .*.: 4
    ehwmol link door1 has different position in instance 1 compared to instance 0. Position difference: [ 1.25909977e+01 -1.06488816e-01 -1.53130677e-04].
    xxipyh link glass has different position in instance 1 compared to instance 0. Position difference: [ 3.22061586e+00 -1.64685936e+01  1.41612031e-02].
    xxipyh link glass has different position in instance 3 compared to instance 0. Position difference: [ 3.22452474e+00 -1.64754295e+01  1.31846406e-02].
    xxipyh link glass has different position in instance 2 compared to instance 0. Position difference: [ 3.22842908e+00 -1.64705448e+01  1.41612012e-02].

  ### Articulated object (.*) instances have different scales for base links. This may have broken things during the match links script.: 3
    Articulated object nwlbit instances have different scales for base links. This may have broken things during the match links script.
    Articulated object rgmujm instances have different scales for base links. This may have broken things during the match links script.
    Articulated object xxipyh instances have different scales for base links. This may have broken things during the match links script.


## scenes/Wainscott_0_garden
  ### Articulated object (.*) instances have different scales for base links. This may have broken things during the match links script.: 11
    Articulated object gpkbiw instances have different scales for base links. This may have broken things during the match links script.
    Articulated object kfmkbm instances have different scales for base links. This may have broken things during the match links script.
    Articulated object xbfgjc instances have different scales for base links. This may have broken things during the match links script.
    Articulated object lsyzkh instances have different scales for base links. This may have broken things during the match links script.
    Articulated object fqhdne instances have different scales for base links. This may have broken things during the match links script.
    Articulated object qohxjq instances have different scales for base links. This may have broken things during the match links script.
    Articulated object emeeke instances have different scales for base links. This may have broken things during the match links script.
    Articulated object lvgliq instances have different scales for base links. This may have broken things during the match links script.
    Articulated object lwjdmj instances have different scales for base links. This may have broken things during the match links script.
    Articulated object pluwfl instances have different scales for base links. This may have broken things during the match links script.
    Articulated object fexqbj instances have different scales for base links. This may have broken things during the match links script.

  ### (.*) link .* has different .* in instance .* compared to instance .*. .* difference: .*.: 3
    inmymj link right_wing has different position in instance 1 compared to instance 0. Position difference: [-0.25      -5.859375  -0.2421875].
    wwhydr link right_wing has different position in instance 2 compared to instance 0. Position difference: [ 0.          1.3125     -0.29296875].
    inmymj link left_wing has different position in instance 1 compared to instance 0. Position difference: [ 0.5        -1.95703125 -0.2421875 ].

  ### (.*) has different pivot offset position \(by (.*)\). Match pivots on each instance.: 2
    window-inmymj-0-right_wing-base_link-P-upper has different pivot offset position (by [ 0.00000000e+00  8.69235475e-05 -1.42415443e-01]). Match pivots on each instance.
    window-inmymj-0-left_wing-base_link-P-upper has different pivot offset position (by [ 2.05363357e-03 -8.69235475e-05  1.42426630e-01]). Match pivots on each instance.


## scenes/Wainscott_0_int
  ### Articulated object (.*) instances have different scales for base links. This may have broken things during the match links script.: 14
    Articulated object fqhdne instances have different scales for base links. This may have broken things during the match links script.
    Articulated object fexqbj instances have different scales for base links. This may have broken things during the match links script.
    Articulated object emeeke instances have different scales for base links. This may have broken things during the match links script.
    Articulated object kfmkbm instances have different scales for base links. This may have broken things during the match links script.
    Articulated object ohagsq instances have different scales for base links. This may have broken things during the match links script.
    Articulated object xbfgjc instances have different scales for base links. This may have broken things during the match links script.
    Articulated object gpkbiw instances have different scales for base links. This may have broken things during the match links script.
    Articulated object lwjdmj instances have different scales for base links. This may have broken things during the match links script.
    Articulated object ktydvs instances have different scales for base links. This may have broken things during the match links script.
    Articulated object lsyzkh instances have different scales for base links. This may have broken things during the match links script.
    Articulated object pluwfl instances have different scales for base links. This may have broken things during the match links script.
    Articulated object ulnafj instances have different scales for base links. This may have broken things during the match links script.
    Articulated object qohxjq instances have different scales for base links. This may have broken things during the match links script.
    Articulated object lvgliq instances have different scales for base links. This may have broken things during the match links script.


## scenes/Wainscott_1_int
  ### Articulated object (.*) instances have different scales for base links. This may have broken things during the match links script.: 9
    Articulated object ktydvs instances have different scales for base links. This may have broken things during the match links script.
    Articulated object xbfgjc instances have different scales for base links. This may have broken things during the match links script.
    Articulated object lvgliq instances have different scales for base links. This may have broken things during the match links script.
    Articulated object bamfsz instances have different scales for base links. This may have broken things during the match links script.
    Articulated object fqhdne instances have different scales for base links. This may have broken things during the match links script.
    Articulated object nnvyol instances have different scales for base links. This may have broken things during the match links script.
    Articulated object ulnafj instances have different scales for base links. This may have broken things during the match links script.
    Articulated object jhymlr instances have different scales for base links. This may have broken things during the match links script.
    Articulated object ohagsq instances have different scales for base links. This may have broken things during the match links script.


In [45]:
from collections import defaultdict
import re

warning_re = re.compile(r"(.*) has too many vertices: (\d+) > 20000")
warn_objs = defaultdict(dict)
for name, x in jsons.items():
    for w in x["warnings"]:
        m = warning_re.fullmatch(w)
        if m:
            warn_objs[name][m.group(1)] = int(m.group(2))
            
print(warn_objs)

defaultdict(<class 'dict'>, {'objects/batch-00': {'L-tag-ohegnp-0': 21794, 'chicken_wire-czoiqw-0': 52305}, 'objects/batch-01': {'L-shoe_rack-spdrzb-0': 59456}, 'objects/batch-02': {'L-radio-vgioak-0': 50123}, 'objects/batch-06': {'zipper-qwsvid-0': 27426}, 'objects/legacy_batch-01': {'swivel_chair-fqbnuk-0-base_link': 36405, 'swivel_chair-lizuow-0-base_link': 25743, 'swivel_chair-lxupys-0-base_link': 35465, 'swivel_chair-mqxrim-0-base_link': 21060, 'swivel_chair-opkoam-0-base_link': 38343}, 'objects/legacy_batch-02': {'table_lamp-pmekkr-0-arm-body-R-lower': 23529, 'table_lamp-pmekkr-0-head-neck-R-lower': 44331, 'table_lamp-pmekkr-0-arm-body-R-upper': 23529, 'table_lamp-pmekkr-0-head-neck-R-upper': 44331, 'table_lamp-pxdhzm-0-head-arm-R-lower': 33474, 'table_lamp-pxdhzm-0-head-arm-R-upper': 33474}, 'objects/legacy_batch-03': {'desk-egzdgz-0-base_link': 20943, 'dishwasher-znfvmj-0-link_0-base_link-P-lower': 31812, 'dishwasher-znfvmj-0-link_1-base_link-P-lower': 31812, 'dishwasher-znfvmj

In [46]:
# Worst offenders
sorted((cnt, obj) for warn_target in warn_objs.values() for obj, cnt in warn_target.items())

[(20082, 'barbecue_sauce_bottle-gfxrnj-0'),
 (20082, 'catsup_bottle-qfvqfm-0'),
 (20082, 'chocolate_sauce_bottle-yegrkf-0'),
 (20082, 'glaze_bottle-zdxagk-0'),
 (20082, 'hot_sauce_bottle-qvpthd-0'),
 (20082, 'mustard_bottle-lgxfyv-0'),
 (20082, 'salsa_bottle-kydilb-0'),
 (20082, 'soy_sauce_bottle-saujjl-0'),
 (20082, 'squeeze_bottle-uzmhdn-0'),
 (20082, 'sugar_syrup_bottle-kdlbbq-0'),
 (20082, 'vanilla_bottle-drevku-0'),
 (20082, 'white_sauce_bottle-gtwngf-0'),
 (20082, 'wine_sauce_bottle-vqtevv-0'),
 (20279, 'pot_plant-kxmvco-0-base_link'),
 (20943, 'desk-egzdgz-0-base_link'),
 (21060, 'swivel_chair-mqxrim-0-base_link'),
 (21794, 'L-tag-ohegnp-0'),
 (22097, 'L-garden_chair-deuron-0'),
 (22097, 'L-garden_chair-deuron-1'),
 (22560, 'room_light-fxtqtn-0'),
 (22560, 'room_light-fxtqtn-1'),
 (22740, 'fridge-jtqazu-0-base_link'),
 (22956, 'L-pedestal_table-ifmhpn-0'),
 (22956, 'L-pedestal_table-ifmhpn-1'),
 (22998, 'C-stockpot-gxiqbw-0'),
 (22998, 'C-stockpot-gxiqbw-1'),
 (23465, 'floor_lam

In [47]:
# Check which of the "articulated object has diff scale" errors are on bad objects
providers = json.loads(pathlib.Path(r"D:\ig_pipeline\artifacts\pipeline\object_inventory.json").read_text())["providers"]
id_providers = {k.split("-")[-1]: v for k, v in providers.items()}
categories = dict([tuple(reversed(k.split("-"))) for k in providers.keys()])
articulation_tmpl = "Articulated object (.*) instances have different scales for base links. This may have broken things during the match links script."
for target, objs in error_matches_by_file[articulation_tmpl].items():
    if not objs:
        continue
    bad_objs = set()
    for match in objs:
        obj = match.group(1)
        if id_providers[obj] != target:
            name = f"{categories[obj]}-{obj}"
            bad_objs.add(name)

    if not bad_objs:
        continue
    
    print(f"{target}: {len(bad_objs)} {bad_objs}")

scenes/Beechwood_0_garden: 6 {'top_cabinet-dmwxyl', 'bottom_cabinet_no_top-qohxjq', 'bottom_cabinet_no_top-spojpj', 'bottom_cabinet-immwzb', 'window-ulnafj', 'door-lvgliq'}
scenes/Beechwood_0_int: 7 {'top_cabinet-dmwxyl', 'bottom_cabinet_no_top-qohxjq', 'bottom_cabinet_no_top-spojpj', 'bottom_cabinet-immwzb', 'window-ulnafj', 'door-ohagsq', 'door-lvgliq'}
scenes/Beechwood_1_int: 9 {'top_cabinet-dmwxyl', 'bottom_cabinet_no_top-pluwfl', 'window-ithrgo', 'bottom_cabinet-jhymlr', 'toilet-kfmkbm', 'bottom_cabinet-kubcdk', 'window-ulnafj', 'door-ohagsq', 'door-lvgliq'}
scenes/Benevolence_0_int: 1 {'door-lvgliq'}
scenes/Benevolence_1_int: 6 {'bottom_cabinet_no_top-ufhpbn', 'top_cabinet-dmwxyl', 'top_cabinet-eobsmt', 'top_cabinet-lsyzkh', 'window-ulnafj', 'cedar_chest-fwstpx'}
scenes/Benevolence_2_int: 2 {'window-ithrgo', 'door-lvgliq'}
scenes/hotel_gym_spa: 2 {'door-ceddpg', 'door-nfrbch'}
scenes/house_single_floor: 1 {'bottom_cabinet_no_top-gjrero'}
scenes/Ihlen_0_int: 3 {'door-ktydvs', 'win

In [48]:
# Get a list of all the objects that show up in the match links relevant errors
mismatched_link_offset_tmpl = "(.*) link .* has different .* in instance .* compared to instance .*. .* difference: .*."
objects_for_match_links = error_matches[mismatched_link_offset_tmpl] | error_matches[articulation_tmpl]
print(", ".join(f'"{x.group(1)}"' for x in sorted(objects_for_match_links, key=lambda x: x.group(1))))

"bamfsz", "bamfsz", "bamfsz", "bamfsz", "bmsclc", "ceddpg", "ceddpg", "ceddpg", "dladgw", "dladgw", "dladgw", "dmwxyl", "dmwxyl", "dmwxyl", "dmwxyl", "dmwxyl", "dmwxyl", "dmwxyl", "dnyzym", "dqnbsj", "dqnbsj", "ehwmol", "ehwmol", "emeeke", "emeeke", "eobsmt", "fexqbj", "fexqbj", "fexqbj", "fqhdne", "fqhdne", "fqhdne", "fwstpx", "fwstpx", "gjrero", "gpkbiw", "gpkbiw", "ijnwlp", "ijnwlp", "ijnwlp", "ijnwlp", "ijnwlp", "ijnwlp", "ijnwlp", "ijnwlp", "ijnwlp", "ijnwlp", "ijnwlp", "immwzb", "immwzb", "immwzb", "inmymj", "inmymj", "ithrgo", "ithrgo", "ithrgo", "ithrgo", "ithrgo", "ithrgo", "ithrgo", "ithrgo", "jhymlr", "jhymlr", "jhymlr", "jrhgeu", "kfmkbm", "kfmkbm", "kfmkbm", "ktydvs", "ktydvs", "ktydvs", "kubcdk", "kwbnhy", "kwbnhy", "kwbnhy", "kwbnhy", "kwbnhy", "lcrefl", "lsyzkh", "lsyzkh", "lsyzkh", "lsyzkh", "lsyzkh", "lsyzkh", "lvgliq", "lvgliq", "lvgliq", "lvgliq", "lvgliq", "lvgliq", "lvgliq", "lvgliq", "lvgliq", "lvgliq", "lvgliq", "lvgliq", "lvgliq", "lvgliq", "lvgliq", "lvgliq", 

In [49]:
import sys
sys.path.append(r"D:\ig_pipeline")
from b1k_pipeline.utils import parse_name

# Get a link of all the objects that are bad cloth
cloth_tmpl = "Cloth object (.*) should consist of exactly 1 element. Currently it has .* elements."
objects_for_cloth = [parse_name(x.group(1)).group('model_id') for x in error_matches[cloth_tmpl]]
print(", ".join(f'"{x}"' for x in sorted(objects_for_cloth)))

"agftpm", "aiftuk", "ajcjzr", "bgvwes", "bpjqwj", "bpjqwj", "bpjqwj", "bpjqwj", "brebdk", "crbvcg", "cuoayr", "dnypbe", "efmfdf", "ekcjci", "eknmzl", "faedft", "feohwy", "fglfga", "ficvss", "fmapwe", "fzldgi", "getwzm", "girtqm", "gtghon", "gxmcpy", "htcikj", "imdhfp", "itrkhr", "ivmsdt", "jfsovh", "jkcugp", "jmujjo", "kclcrj", "kiiium", "klhkgd", "laksie", "ldpbcu", "lqweda", "mksdlu", "mpxtgf", "mxxyva", "njidcf", "nmvvil", "nogevo", "nowqqh", "nzbhjq", "ohvomi", "ohvomi", "ojgwec", "omsbux", "padfds", "pbytey", "pchzob", "prhems", "pvzxyp", "qewdqa", "qofxar", "rcgdde", "remcyk", "rhgweu", "rygqoc", "sehjcp", "shbakk", "shbakk", "stcrad", "sullco", "szdonp", "szdonp", "szdonp", "szdonp", "tfnwti", "thmepr", "tnirgd", "trwest", "ttgike", "uaidmc", "uksirc", "uksirc", "uwzqxm", "uxgjdv", "vfknnl", "vpafgj", "vqbvph", "wbhliu", "wczokr", "xeskbh", "xzuywj", "ydgivr", "ykiiqr", "ylycdz"
